In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 4.2 MB 23.1 MB/s eta 0:00:01
     |████████████████████████████████| 6.6 MB 34.8 MB/s eta 0:00:01
     |████████████████████████████████| 86 kB 41.3 MB/s eta 0:00:01


In [2]:
# Load gpt2-model and tokenizer

In [2]:
import torch

PATH_model = '../Model/output/GPT-2 Model/gpt2-large-model.pth'
PATH_tok = "../Model/output/GPT-2 Model/gpt2-large-tokenizer.pth"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = torch.load(PATH_model).to(device)
tokenizer= torch.load(PATH_tok)

In [3]:
'''
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-large') 
model = GPT2LMHeadModel.from_pretrained('gpt2-large' , pad_token_id = tokenizer.eos_token_id)
'''

"\ntokenizer = GPT2Tokenizer.from_pretrained('gpt2-large') \nmodel = GPT2LMHeadModel.from_pretrained('gpt2-large' , pad_token_id = tokenizer.eos_token_id)\n"

In [4]:
'''
import torch

model_PATH = '../Model/output/GPT-2 Model/gpt2-xl-model.pth'
toc_PATH = '../Model/output/GPT-2 Model/gpt2-xl-tokenizer.pth'

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = torch.load(model_PATH).to(device)
tokenizer = torch.load(toc_PATH)
'''

'\nimport torch\n\nmodel_PATH = \'../Model/output/GPT-2 Model/gpt2-xl-model.pth\'\ntoc_PATH = \'../Model/output/GPT-2 Model/gpt2-xl-tokenizer.pth\'\n\ndevice = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")\nmodel = torch.load(model_PATH).to(device)\ntokenizer = torch.load(toc_PATH)\n'

In [5]:
# Read DataFrame From csv file

In [30]:
import pandas as pd

df = pd.read_csv('news_for_generation.csv')
df = df.iloc[8500:9000]

In [31]:
df.shape

(500, 5)

In [26]:
# Create a generator function

In [32]:
def fakenews_generator(df, tokenizer, model, file_name, list):
    from datetime import datetime
    START = datetime.now()
    print(len(df))
    i = 1
    c = 0
    for index, row in df.iterrows():
        d={}
        sentence = row['title']
        input_ids = tokenizer.encode(sentence, return_tensors='pt').to(device)
        output = model.generate(input_ids, max_length=1000, num_beams=5, no_repeat_ngram_size=2, early_stopping=True)
        fake_news = tokenizer.decode(output[0], skip_special_tokens=True).replace(sentence, '')
        c += 1
        if i%100 == 0:
            print(str(i)+'/'+str(len(df)),' || Time: ' + str(datetime.now() - START), 'Total steps: ' + str(c))
        if len(str(fake_news.split())) >= 128:
            try:
                d['Real title'] = row['title']
                d['Generated text'] = fake_news.replace('\n\n', '')
                d['label'] = 2
                list.append(d)
            except:
                d['Real title'] = row['title']
                d['Generated text'] = fake_news
                d['label'] = 2
                list = list.append(d)
            i += 1
        else:
            pass
    pd.DataFrame(list).to_csv('Dataset/generated_article' + str(file_name) +'.csv', index=False)
    print(datetime.now() - START)

In [33]:
if __name__ == '__main__':
    file_name = 11
    list = []
    fakenews_generator(df, tokenizer, model, file_name, list)

500
100/500  || Time: 0:22:26.118572 Total steps: 111
200/500  || Time: 0:51:20.714220 Total steps: 217
300/500  || Time: 1:16:46.485315 Total steps: 329
400/500  || Time: 1:40:08.130879 Total steps: 438
1:53:44.311804


In [12]:
import pandas as pd
df_test = pd.read_csv('Dataset/generated_article9.csv')

In [13]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 462 entries, 0 to 461
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Real title      462 non-null    object
 1   Generated text  462 non-null    object
 2   label           462 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 11.0+ KB


In [22]:
c = 0
for index, text in df_test.iterrows():
    if len(str(text['Generated text']).split())< 50:
        c += 1
        
c

99

In [19]:
'''
c = 0
list_test = []
for index, row in df_test.iterrows():
    d = {}
    if len(str(row['Generated text'])) >= 128:
        c += 1
        print(row['Generated text'])
        print('##################')
        break
    else:
        pass
    
print(c)
'''

"\nc = 0\nlist_test = []\nfor index, row in df_test.iterrows():\n    d = {}\n    if len(str(row['Generated text'])) >= 128:\n        c += 1\n        print(row['Generated text'])\n        print('##################')\n        break\n    else:\n        pass\n    \nprint(c)\n"

In [4]:
'''
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained("gpt2-xl")
model = GPT2LMHeadModel.from_pretrained("gpt2-xl")
'''

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/689 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.99G [00:00<?, ?B/s]

In [83]:
# pd.DataFrame(list_test).to_csv("Dataset/generated_article.csv")

In [ ]:
# df_test['Generated text'].iloc[25].replace('\n\n', '').isnull()